In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class":'a-price a-text-normal aok-align-center reinventPriceAccordionT2'}).find("span", attrs={"class": "a-offscreen"}).text

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={"class":'a-price a-text-normal aok-align-center reinventPriceAccordionT2'}).find("span", attrs={"class": "a-offscreen"}).text

        except:
            price = ""

    return price

#Function to get ingredients
def get_ig(soup):
  try:
    description = soup.find("div", attrs={"id": "productDescription"}).text
    # Find the substring "Ingredients:"
    start_index = description.find("Ingredients") + len("Ingredients")
    # Find the next occurrence of "." after the start index
    end_index = description.find("Directions", start_index)
    # Extract the ingredients substring
    ingredients = description[start_index:end_index]
  except AttributeError:
    ingredients = ""
  return ingredients


def get_category_and_type(soup):

  try:
    # Find the product details section
    details = soup.find(id="detailBullets_feature_div")

    # Find the category and type elements
    category_element = details.find(text="Department")
    type_element = details.find(text="Product Type")

    # Extract the category and type values
    category = category_element.find_next_sibling("span").text.strip()
    product_type = type_element.find_next_sibling("span").text.strip()

    return category, product_type
  except AttributeError:
    return None, None
def get_product_type(url):
  url=str(url)
  if "body+wash" in url:
    return "Body Wash"
  elif "body+oil" in url:
    return "Body Oil"
  elif "body+serum" in url:
    return "Body Serum"
  elif "body+lotion" in url:
    return "Body Lotion"
  elif "sunscreen" in url:
    return "Sunscreen"
  elif "body+scrub" in url:
    return "Body Scrub"
  elif "body+gel" in url:
    return "Body Gel"
  else:
    return None

In [ ]:

if __name__ == '__main__':

    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    # URL1 = "https://www.amazon.co.uk/s?k=sunscreen&crid=1S6GNCWW7B7N4&sprefix=sunscree%2Caps%2C382&ref=nb_sb_noss_2"
    url_list = ["https://www.amazon.co.uk/s?k=body+oil&crid=RWF492K2AN66&sprefix=body+oi%2Caps%2C405&ref=nb_sb_noss_2",
                "https://www.amazon.co.uk/s?k=sunscreen&crid=1S6GNCWW7B7N4&sprefix=sunscree%2Caps%2C382&ref=nb_sb_noss_2",
                "https://www.amazon.co.uk/s?k=body+scrub&crid=250VJ9JR313U9&sprefix=body+scrub%2Caps%2C424&ref=nb_sb_noss_1",
                "https://www.amazon.co.uk/s?k=body+wash&crid=OM33DRWOWOD4&sprefix=body+wash%2Caps%2C306&ref=nb_sb_noss_1",
                "https://www.amazon.co.uk/s?k=body+lotion&crid=1DBBA0Y80QMXS&sprefix=body+lotio%2Caps%2C297&ref=nb_sb_noss_2",
                "https://www.amazon.co.uk/s?k=body+serum&crid=2TB9GH5TQO6YG&sprefix=body+serum%2Caps%2C295&ref=nb_sb_noss_1",
                "https://www.amazon.co.uk/s?k=body+gel&crid=DIUTN0YD7OHT&sprefix=body+ge%2Caps%2C309&ref=nb_sb_noss_2",
                ]

    d = {"title":[], "price":[], "URL":[],"Ingredients":[],"Category":[],"Product Type":[]}
    # HTTP Request
    for URL in url_list:
      webpage = requests.get(URL, headers=HEADERS)

      # Soup Object containing all data
      soup = BeautifulSoup(webpage.content, "html.parser")

      # Fetch links as List of Tag Objects
      links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

      # Store the links
      links_list = []

      # Loop for extracting links from Tag Objects
      for link in links:
              links_list.append(link.get('href'))

      # Loop for extracting product details from each link
      for link in links_list[:1]:
          new_webpage = requests.get("https://www.amazon.co.uk" + link, headers=HEADERS)

          new_soup = BeautifulSoup(new_webpage.content, "html.parser")

          # Function calls to display all necessary product information
          d['title'].append(get_title(new_soup))
          d['price'].append(get_price(new_soup))
          d['URL'].append("www.amazon.co.uk"+link)
          d['Ingredients'].append(get_ig(new_soup))
          d["Category"] = get_category_and_type(new_soup)[0]
          d["Product Type"].append(get_product_type(URL))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


<ipython-input-28-ef38c68a381e>:58: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  category_element = details.find(text="Department")
<ipython-input-28-ef38c68a381e>:59: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  type_element = details.find(text="Product Type")


In [ ]:
amazon_df

,title,price,URL,Ingredients,Category,Product Type
0,"Aveeno Skin Relief Body Oil Spray, With Oat Oi...",£5.80,www.amazon.co.uk/Aveeno-Skin-Relief-Spray-200m...,"Glycine Soja Oil, Avena Sativa Kernel (Oat)...",None,Body Oil
1,Garnier Ambre Solaire Hydra 24 Hour Protect Lo...,£4.99,www.amazon.co.uk/Garnier-Resistant-Sunscreen-P...,"884841 4,Aqua / Water,Isopropyl Palmitate,G...",None,Sunscreen
2,Organic Shop Hydrating Body Scrub Coconut (250ml),£3.74,www.amazon.co.uk/Organic-Shop-Hydrating-Scrub-...,"Sucrose, Glycerin, Cetearyl Alcohol, Butyro...",None,Body Scrub
3,Dove Relaxing microbiome-gentle Body Wash for ...,£1.25,www.amazon.co.uk/Dove-Relaxing-Body-wash-225/d...,"Aqua, Cocamidopropyl Betaine, Sodium Lauroy...",None,Body Wash
4,"Cetaphil Face & Body Moisturiser, 473ml, Moist...",£10.62,www.amazon.co.uk/sspa/click?ie=UTF8&spc=MTo0Nz...,Aqua Glycerin Isopropyl Palmitate Cetearyl ...,None,Body Lotion
5,BYOMA BODY Smoothing Body Serum 200ml,£9.99,www.amazon.co.uk/BYOMA-BODY-Smoothing-Serum-20...,Aqua (Water) Isopropyl Myristate Butylene G...,None,Body Serum
6,"Baylis & Harding Goodness Oud, Cedar and Amber...",£9.97,www.amazon.co.uk/Baylis-Harding-Goodness-Cedar...,"Aqua (Water), Cocamidopropyl Betaine, Sodiu...",None,Body Gel
